In [1]:
import pandas as pd
import json
from unidecode import unidecode

In [2]:
def trat_nome(nome):
    nome_trat = unidecode(nome).upper().strip()
    return nome_trat

In [29]:
# bases
candidatos_hist = pd.read_excel('./db/deps_federais/deps_federais_2010_2022.xlsx')
candidatos_2022 = candidatos_hist[candidatos_hist['ANO_ELEICAO'] == 2022].reset_index()

eleitos = pd.read_excel('./db/eleicoes.xlsx')

df_parls = pd.read_excel('./db/deps_federais/deputados.xlsx')
df_parls_56 = df_parls[df_parls['idLegislaturaFinal'] == 56].reset_index()

with open('./db/deps_federais/parlamentares.json', 'r', encoding="UTF-8") as f:
    parls_ativos = json.load(f)
df_parls_ativos = pd.DataFrame(parls_ativos)

d:\Python\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [30]:
# tratar nomes
candidatos_2022['nome_urna_trat'] = candidatos_2022['NM_URNA_CANDIDATO'].apply(trat_nome)
candidatos_2022['nome_trat'] = candidatos_2022['NM_CANDIDATO'].apply(trat_nome)

eleitos['nome_trat'] = eleitos['nome'].apply(trat_nome)

df_parls_56['nome_civil_trat'] = df_parls_56['nomeCivil'].apply(trat_nome)
df_parls_56['nome_trat'] = df_parls_56['nome'].apply(trat_nome)
df_parls_56['id'] = df_parls_56['uri'].apply(lambda x: int(x.split('/')[-1]))

In [31]:
# base precisou ser tratada manualmente
eleitos_merged = eleitos.merge(candidatos_2022[['nome_urna_trat', 'ST_REELEICAO']], left_on='nome_trat', right_on='nome_urna_trat', how='left')
eleitos_merged.to_excel('./db/eleitos_merged.xlsx')
eleitos_trat = pd.read_excel('./db/eleitos_merged_trat.xlsx')

RENOVAÇÃO

In [32]:
def is_candidate(nome):
    cand_n_urna = list(candidatos_2022['nome_urna_trat'])
    cand_n = list(candidatos_2022['nome_trat'])
    return nome in cand_n or nome in cand_n_urna

In [33]:
# dados mensurar withdrawal

df_parls_ativos_merged = df_parls_ativos.merge(df_parls_56[['id', 'nome_civil_trat']], on='id')
df_parls_ativos_merged['is_cand'] = df_parls_ativos_merged['nome_civil_trat'].apply(is_candidate)

In [34]:
# # dados para mensurar defeated (HÁ PROBLEMAS NO BATIMETO COM OS NOMES... POR ISSO VOU USAR A BASE TRATADA MANUALMENTE)
# df_parls_ativos_cand = df_parls_ativos_merged[df_parls_ativos_merged['is_cand']].reset_index(drop=True)
# df_parls_ativos_cand['nome_trat'] = df_parls_ativos_cand['nome'].apply(trat_nome)
# df_parls_ativos_cand['is_elected'] = df_parls_ativos_cand['nome_trat'].apply(lambda x: x in list(eleitos['nome_trat']))

In [35]:
n = len(eleitos_trat[eleitos_trat['ST_REELEICAO'] == 'N'])
r = len(eleitos_trat[eleitos_trat['ST_REELEICAO'] == 'S'])
w = len(df_parls_ativos_merged[~df_parls_ativos_merged['is_cand']])
t = len(eleitos_trat)

In [36]:
renov_bruta = n / (n + r)
renov_com = w / t


In [37]:
d = t * (renov_bruta - renov_com)

In [42]:
renov_liq = d / (d + r)
conservacao = r / (d + r)

In [43]:
print(f"""
Renovação bruta: {round(renov_bruta * 100, 2)}
Renovação compulsória: {round(renov_com * 100, 2)}
Renovação líquida: {round(renov_liq * 100, 2)}
Conservação: {round(conservacao * 100 ,2)}
""")


Renovação bruta: 44.25
Renovação compulsória: 14.42
Renovação líquida: 34.85
Conservação: 65.15



In [40]:
t

513